In [ ]:
import os
import pandas as pd
import json
from pathlib import Path

# 1. THE UNIVERSAL WAY: Get the current directory 
try:
    # Works as a script (.py)
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    # Works in a Notebook (.ipynb)
    BASE_DIR = Path.cwd()

# 2. ADAPTED NAVIGATION:
# .parent moves up from 'mcp_tools_development' to 'ai-engeneering-study-mcp'
# Then we join into 'data_prepared' and use the plural 'events.json'
INPUT_FILE = BASE_DIR.parent / 'data_prepared' / 'events.json'

# Simple check to verify
if INPUT_FILE.exists():
    print(f"✅ Setup complete. Input file found: {INPUT_FILE}")
else:
    print(f"❌ Warning: Input file NOT found at {INPUT_FILE}")
    # Debugging: show what BASE_DIR actually is
    print(f"Current BASE_DIR: {BASE_DIR}")

In [ ]:
# Load the raw JSON
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# 1. Load the main data into a DataFrame
df = pd.DataFrame(raw_data)

# 2. Flatten ONLY the 'eventGuests' column
# This turns the dictionary into a mini-dataframe
guests_df = pd.json_normalize(df['eventGuests'])

# 3. Rename columns to keep the 'eventGuests_' prefix for clarity
guests_df.columns = [f"eventGuests_{col}" for col in guests_df.columns]

# 4. Join the new columns back to the original data and drop the old dict column
events_df = pd.concat([df.drop(columns=['eventGuests']), guests_df], axis=1)

print(f"✅ Successfully processed {len(events_df)} records.")
print(f"Flattened columns: {list(guests_df.columns)}")

# Display result
display(events_df.head(3))

In [ ]:
# Convert date string to actual datetime objects
events_df['date'] = pd.to_datetime(events_df['date'], errors='coerce')

# Preview the current state of the columns
print("--- Final Column List ---")
print(events_df.columns.tolist())

print("\n--- Data Types Check ---")
print(events_df.dtypes[['date', 'eventGuests_total', 'eventGuests_going']])

display(events_df.head())

In [ ]:
# How many events all time?
total_events = len(events_df)

print(f"Total events in database: {total_events}")

In [ ]:
# Which year has the most events organized?

# Count events per year
yearly_stats = events_df['date'].dt.year.value_counts()

# Identify the top year
top_year = yearly_stats.idxmax()
max_events = yearly_stats.max()

print(f"The most productive year was {top_year} with {max_events} events.")

# Optional: Show the full breakdown
print("\Events per year:")
print(yearly_stats)


In [ ]:
# How many events in a given year?
target_year = 2024
events_in_year = events_df[events_df['date'].dt.year == target_year]

print(f"Events in {target_year}: {len(events_in_year)}")

In [ ]:
# 1. Total Attendance across all time
total_all_time = events_df['eventGuests_going'].sum()

# 2. Filtering for a specific year and finding the "Star" event
year_2024 = events_df[events_df['date'].dt.year == 2024]
all_participants = year_2024['eventGuests_going'].sum()
most_popular = year_2024.loc[year_2024['eventGuests_total'].idxmax()]

print(f"All time event participants: {total_all_time}")
print(f"The number of all participants of the events in 2024: {all_participants}")
print(f"The most popular event in 2024 was: {most_popular['title']}")
print(f"Attendance: {most_popular['eventGuests_total']}")


In [ ]:
# How many online or venue events in all time?
location_online = "ONLINE"
location_venue = "VENUE"

# Create a mask with two conditions using the & (AND) operator
mask_online = (events_df['location'] == location_online)
mask_venue = (events_df['location'] == location_venue)

online_count = len(events_df[mask_online])
venue_count = len(events_df[mask_venue])

print(f"There were {online_count} {location_online} events in all time.")
print(f"There were {venue_count} {location_venue} events in all time.")

In [ ]:
# How many online or venue events in a given year?
year = 2025
location_online = "ONLINE"
location_venue = "VENUE"

# Create a mask with two conditions using the & (AND) operator
mask_online = (events_df['date'].dt.year == year) & (events_df['location'] == location_online)
mask_venue = (events_df['date'].dt.year == year) & (events_df['location'] == location_venue)

online_count = len(events_df[mask_online])
venue_count = len(events_df[mask_venue])

print(f"There were {online_count} {location_online} events in {year}.")
print(f"There were {venue_count} {location_venue} events in {year}.")

In [ ]:
# Most visited events all time?
# Sort by total guests descending, take the top 3
top_3_all = events_df.sort_values(by='eventGuests_going', ascending=False).head(3)

# Display only title, date, and total for readability
display(top_3_all[['title', 'date', 'eventGuests_going']])

In [ ]:
# Most visited events in a given year?
year = 2024
top_3_year = events_df[events_df['date'].dt.year == year].sort_values(
    by='eventGuests_total', 
    ascending=False
).head(3)

display(top_3_year[['title', 'eventGuests_total']])

In [ ]:
# How many events for a given category in all time?
category_simonton_club = "simontonklub"
category_simonton_workshop = "simontonösszehangolva"

# Create a mask with two conditions using the & (AND) operator
total_all_time_simonton_club = (events_df['categories'] == category_simonton_club).sum()
total_all_time_simonton_workshop = (events_df['categories'] == category_simonton_workshop).sum()

print(f"There were {total_all_time_simonton_club} Simonton club events in all time.")
print(f"There were {total_all_time_simonton_workshop} Simonton workshop events in all time.")

In [ ]:
# How many events for a given category in a given year?
year = 2025
category_simonton_club = "simontonklub"
category_simonton_workshop = "simontonösszehangolva"

# Create a mask with two conditions using the & (AND) operator
mask_simonton_club = (events_df['date'].dt.year == year) & (events_df['categories'] == category_simonton_club)
mask_simonton_workshop = (events_df['date'].dt.year == year) & (events_df['categories'] == category_simonton_workshop)

simonton_club_count = len(events_df[mask_simonton_club])
simonton_workshop_count = len(events_df[mask_simonton_workshop])

print(f"There were {simonton_club_count} Simonton club events in {year}.")
print(f"There were {simonton_workshop_count} Simonton workshop events in {year}.")

In [ ]:
# Most visited events for a given category in all time?
# 1. Filter for Club events
simonton_club_df = events_df[events_df['categories'] == category_simonton_club]
total_all_time_simonton_club = len(simonton_club_df)

print(f"There were {total_all_time_simonton_club} Simonton club events in all time.")

# 2. Sort and display top 3 Club events
top_3_all_simonton_club = simonton_club_df.sort_values(by='eventGuests_going', ascending=False).head(3)
display(top_3_all_simonton_club[['title', 'date', 'eventGuests_going']])

In [ ]:
# Most visited events for a given category in a given year?
# 1. Filter for Workshop events
year = 2025
category_simonton_workshop = "simontonösszehangolva"

# Create the mask
mask = (events_df['date'].dt.year == year) & (events_df['categories'] == category_simonton_workshop)

# Apply the mask to the DataFrame to get ONLY those rows
filtered_workshops = events_df[mask]

# Now you can count them using len() on the filtered DataFrame
count_workshops = len(filtered_workshops)
print(f"There were {count_workshops} Simonton workshop events in {year}.")

# 2. Sort and display top 3 Workshop events
# We use 'filtered_workshops' because it is still a DataFrame
top_3_year_simonton_workshop = filtered_workshops.sort_values(by='eventGuests_going', ascending=False).head(3)

display(top_3_year_simonton_workshop[['title', 'date', 'eventGuests_going']])

In [ ]:
# Which events had the highest "Waitlist" count?
# Sort by waitlist count in descending order
high_demand_events = events_df.sort_values(by='eventGuests_waitlist', ascending=False)

# Display the top 5 events with the most people on the waitlist
print("Top 5 High-Demand Events (Waitlist):")
display(high_demand_events[['title', 'date', 'eventGuests_waitlist', 'categories']].head(5))

In [ ]:
# 1. High-Demand Events (Highest Waitlist) in a Given Year
target_year = 2025

# Filter for the year and sort by waitlist
high_demand_year = events_df[events_df['date'].dt.year == target_year].sort_values(by='eventGuests_waitlist', ascending=False)

print(f"Top 5 High-Demand Events in {target_year} (Waitlist):")
display(high_demand_year[['title', 'date', 'eventGuests_waitlist', 'categories']].head(5))

# 2. Category Popularity Ranking within a Given Year
# This identifies which categories were most successful in a specific year
target_year = 2025

# Filter for the year
year_data = events_df[events_df['date'].dt.year == target_year]

# Group by category to see performance
category_ranking = year_data.groupby('categories').agg(
    total_events=('id', 'count'),
    total_attendees=('eventGuests_going', 'sum'),
    avg_attendees=('eventGuests_going', 'mean')
).sort_values(by='total_attendees', ascending=False)

print(f"\nCategory Popularity Ranking for {target_year}:")
display(category_ranking)